In [6]:
!pip install biopython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 23.5 MB/s eta 0:00:00


In [7]:
import csv
import pandas as pd
from Bio import SeqIO
from collections import defaultdict
import numpy as np
import re

In [8]:
pan_og_874 = pd.read_csv('assigned_orthogroups_874.csv', na_values=['NaN'])
pan_og_874['gene_id'] = pan_og_874['gene_id'].str.replace(r'^874_', '', regex=True)
pan_og_914 = pd.read_csv('assigned_orthogroups_914.csv', na_values=['NaN'])
pan_og_914['gene_id'] = pan_og_914['gene_id'].str.replace(r'^914_', '', regex=True)
mmseq_clu = pd.read_csv('DB_clu.tsv', delimiter ='\t', names=['Representative', 'Cluster_Members'])
filtered_874 = list(SeqIO.parse("filtered_transcripts_874.fasta", "fasta"))
filtered_914 = list(SeqIO.parse("filtered_transcripts_914.fasta", "fasta"))

In [22]:
mmseq_clu

,Representative,Cluster_Members
0,NODE_100000_length_314_cov_171.430189_g54640_i0,NODE_100000_length_314_cov_171.430189_g54640_i0
1,NODE_100144_length_312_cov_391.022814_g54784_i0,NODE_100144_length_312_cov_391.022814_g54784_i0
2,NODE_100297_length_310_cov_1002.049808_g54937_i0,NODE_100297_length_310_cov_1002.049808_g54937_i0
3,NODE_100484_length_308_cov_1823.718147_g55124_i0,NODE_100484_length_308_cov_1823.718147_g55124_i0
4,NODE_100484_length_308_cov_1823.718147_g55124_i0,NODE_94513_length_385_cov_1181.008929_g49155_i0
...,...,...
47142,NODE_126287_length_185_cov_1.948529_g80926_i0,NODE_126287_length_185_cov_1.948529_g80926_i0
47143,NODE_126482_length_185_cov_0.808824_g81121_i0,NODE_126482_length_185_cov_0.808824_g81121_i0
47144,NODE_126643_length_185_cov_0.698529_g81282_i0,NODE_126643_length_185_cov_0.698529_g81282_i0
47145,NODE_126795_length_184_cov_585.814815_g81434_i0,NODE_126795_length_184_cov_585.814815_g81434_i0


## Assign OGs to the MMSeq2 predited proteins that did not get an OG from the pan genome

In [9]:
orthogroup_dict = {}
t_counter = 1
u_counter = 1

for rep, members in mmseq_clu.groupby('Representative')['Cluster_Members']:
    if len(members) > 1:
        og_name = f"GEP_t_{t_counter:05d}"
        t_counter += 1
    else:
        og_name = f"GEP_u_{u_counter:05d}"
        u_counter += 1
    for member in members:
        orthogroup_dict[member] = og_name

mmseq_ogs = pd.DataFrame(list(orthogroup_dict.items()), columns=['gene_id', 'assigned_orthogroup'])
mmseq_ogs.to_csv('mmseq_ogs.csv', index=False)

### Sanity Check

In [7]:
#assigned_df.tail(50)
og_counts = mmseq_ogs.groupby('assigned_orthogroup').size()
# Count how many OGs have 1, 2, 3, etc. transcripts
og_summary = og_counts.value_counts().sort_index()
print(og_summary)

1     38396
2      2946
3       517
4       153
5        48
6        15
7        13
8         7
9         1
10        2
11        3
12        2
13        1
16        2
17        1
29        1
42        1
Name: count, dtype: int64


In [10]:
strain_mapping = defaultdict(list)

# Parse all of the transcripts for RCC874
for record in SeqIO.parse("filtered_transcripts_874.fasta", "fasta"):
    gene_id_from_fasta = record.id.strip()
    strain_mapping[gene_id_from_fasta].append('874')
for record in SeqIO.parse("filtered_transcripts_914.fasta", "fasta"):
    gene_id_from_fasta = record.id.strip()
    strain_mapping[gene_id_from_fasta].append('914') 

#Convert lists to comma-separated strings for strains with multiple sources
for gene_id, strains in strain_mapping.items():
    if len(set(strains)) > 1:
        strain_mapping[gene_id] = 'Both'  # Assign 'Both' if it contains both strains
    else:
        strain_mapping[gene_id] = ', '.join(strains)

# Map the strains to mmseq_ogs DataFrame
mmseq_ogs['strain'] = mmseq_ogs['gene_id'].map(strain_mapping)

# Count how many are 914 and how many are 874
strain_counts = mmseq_ogs['strain'].str.get_dummies(sep=', ').sum()
print("\nCounts of each strain:\n", strain_counts)


Counts of each strain:
 874    22043
914    23918
[]      1186
dtype: int64


In [11]:
#Remove bacteria predicted proteins
mmseq_ogs = mmseq_ogs[(mmseq_ogs['strain'] == '874') | (mmseq_ogs['strain'] == '914')]
strain_counts_filtered = mmseq_ogs['strain'].str.get_dummies(sep=', ').sum()
print("\nCounts of each strain after filtering:\n", strain_counts_filtered)


Counts of each strain after filtering:
 874    22043
914    23918
dtype: int64


## Assign OGs for non-coding sequences

In [13]:
transcripts_874 = [record.id for record in SeqIO.parse("filtered_transcripts_874.fasta", "fasta")]
transcripts_914 = [record.id for record in SeqIO.parse("filtered_transcripts_914.fasta", "fasta")]
transcriptID_874 = set(transcripts_874)
transcriptID_914 = set(transcripts_914)
mmseq2_gene_ids = set(mmseq_ogs['gene_id'])
gene_ids_874 = set(pan_og_874['gene_id'])
gene_ids_914 = set(pan_og_914['gene_id'])

all_transcripts = transcriptID_874 | transcriptID_914
# Remove transcripts found in gene_ids_874, gene_ids_914, or mmseq2_gene_ids
noncoding_transcripts = all_transcripts - (gene_ids_874 | gene_ids_914)
noncoding_ogs = pd.DataFrame(list(noncoding_transcripts), columns=['gene_id'])

noncoding_ogs['assigned_orthogroup'] = [f'GEP_n_{i+1:05d}' for i in range(len(noncoding_ogs))]
noncoding_ogs.to_csv('noncoding_ogs.csv', index=False)

## Filter for just the OGs in the pan genome

In [14]:
# Replace 'NaN' and 'nan' with actual NaN values
pan_og_874.replace({'assigned_orthogroup': ['NaN']}, np.nan, inplace=True)
pan_og_914.replace({'assigned_orthogroup': ['NaN']}, np.nan, inplace=True)

filtered_874 = pan_og_874.dropna(subset=['assigned_orthogroup'])
filtered_914 = pan_og_914.dropna(subset=['assigned_orthogroup'])

combined_ogs = pd.concat([filtered_874, filtered_914], ignore_index=True)

final_combined_ogs = combined_ogs[['gene_id', 'assigned_orthogroup']]
final_combined_ogs.to_csv('combined_assigned_orthogroups.csv', index=False)

In [15]:
final_combined_ogs

,gene_id,assigned_orthogroup
0,NODE_100003_length_218_cov_1.508876_g61485_i0,GEP_3_36410
1,NODE_100007_length_218_cov_1.455621_g61489_i0,GEP_5_83366
2,NODE_10000_length_2849_cov_6812.425357_g3011_i1,GEP_2_10045
3,NODE_10001_length_2849_cov_465.148214_g3009_i1,GEP_3_40697
4,NODE_100021_length_218_cov_1.183432_g61503_i0,GEP_S_193982
...,...,...
162656,NODE_99999_length_314_cov_184.928302_g54639_i0,GEP_S_059213
162657,NODE_9999_length_2647_cov_624.535412_g3749_i0,GEP_1_07449
162658,NODE_999_length_4939_cov_507.181800_g441_i0,GEP_1_02506
162659,NODE_99_length_8510_cov_337.240397_g51_i0,GEP_1_02709


## Combine everything together

In [16]:
mmseq_ogs = pd.read_csv('mmseq_ogs.csv')
final_combined_ogs = pd.read_csv('combined_assigned_orthogroups.csv')
noncoding_ogs = pd.read_csv('noncoding_ogs.csv')

# Merging dataframes
merged_data = pd.merge(mmseq_ogs, final_combined_ogs, on='gene_id', how='outer', suffixes=('_mmseq', '_final'))
merged_data['assigned_orthogroup'] = merged_data['assigned_orthogroup_mmseq'].combine_first(merged_data['assigned_orthogroup_final'])
merged_data = merged_data.drop(columns=['assigned_orthogroup_mmseq', 'assigned_orthogroup_final'])
merged_data = pd.merge(merged_data, noncoding_ogs, on='gene_id', how='outer', suffixes=('_combined', '_noncoding'))
merged_data['assigned_orthogroup'] = merged_data['assigned_orthogroup_combined'].combine_first(merged_data['assigned_orthogroup_noncoding'])
merged_data = merged_data.drop(columns=['assigned_orthogroup_combined', 'assigned_orthogroup_noncoding'])

In [17]:
merged_data

,gene_id,assigned_orthogroup
0,NODE_100000_length_218_cov_1.508876_g61482_i0,GEP_n_97879
1,NODE_100000_length_314_cov_171.430189_g54640_i0,GEP_u_00001
2,NODE_100001_length_218_cov_1.508876_g61483_i0,GEP_n_41643
3,NODE_100001_length_314_cov_161.883019_g54641_i0,GEP_3_36516
4,NODE_100002_length_218_cov_1.508876_g61484_i0,GEP_n_89144
...,...,...
308506,NODE_999_length_5580_cov_500.866751_g457_i0,GEP_1_05422
308507,NODE_99_length_8510_cov_337.240397_g51_i0,GEP_1_02709
308508,NODE_99_length_9924_cov_150.905114_g61_i0,GEP_1_08639
308509,NODE_9_length_14200_cov_203.386404_g0_i1,GEP_3_38440


### Adding strain ID back

In [18]:
strain_mapping = defaultdict(list)

for record in SeqIO.parse("filtered_transcripts_874.fasta", "fasta"):
    gene_id_from_fasta = record.id.strip()
    strain_mapping[gene_id_from_fasta].append('874')
for record in SeqIO.parse("filtered_transcripts_914.fasta", "fasta"):
    gene_id_from_fasta = record.id.strip()
    strain_mapping[gene_id_from_fasta].append('914')

merged_data['strain'] = merged_data['gene_id'].map(lambda gene_id: ''.join(strain_mapping.get(gene_id, 'Unknown')))

if 'strain' in merged_data.columns:
    # Fill any remaining missing strain values with 'Unknown'
    merged_data['strain'] = merged_data['strain'].replace('', 'Unknown')
else:
    print("The 'strain' column was not created correctly.")

# Count the occurrences of each strain
strain_counts = merged_data['strain'].value_counts(dropna=False)
print("\nCounts of each strain:\n", strain_counts)


Counts of each strain:
 strain
914        156717
874        150449
Unknown      1345
Name: count, dtype: int64


### Remove bacterial sequences

In [19]:
unknown_strains = merged_data[merged_data['strain'] == 'Unknown']

# Display the gene IDs associated with 'Unknown' strains
if not unknown_strains.empty:
    print(f"Found {len(unknown_strains)} gene_ids with 'Unknown' strains:")
    print(unknown_strains['gene_id'].head())  # Show the first few gene IDs
else:
    print("No gene_ids with 'Unknown' strains found.")

# Remove more bacterial transcripts
merged_data = merged_data[merged_data['strain'] != 'Unknown']
merged_data.reset_index(drop=True, inplace=True)

merged_data.to_csv('final_orthogroups.csv', index=False)

Found 1345 gene_ids with 'Unknown' strains:
471    NODE_100218_length_217_cov_5.130952_g61700_i0
496    NODE_100230_length_217_cov_3.642857_g61712_i0
503    NODE_100234_length_217_cov_3.321429_g61716_i0
510    NODE_100238_length_217_cov_3.065476_g61720_i0
519    NODE_100242_length_311_cov_8.946565_g54882_i0
Name: gene_id, dtype: object


### Sanity Check

In [20]:
gep_pattern_count = {
    'GEP_t': merged_data['assigned_orthogroup'].str.count(r'GEP_t\w+').sum(),
    'GEP_u': merged_data['assigned_orthogroup'].str.count(r'GEP_u\w+').sum(),
    'GEP_n': merged_data['assigned_orthogroup'].str.count(r'GEP_n\w+').sum(),
}

total_assigned = len(merged_data['assigned_orthogroup'])
total_known_patterns = sum(gep_pattern_count.values())
gep_pattern_count['pan_OG'] = total_assigned - total_known_patterns

print(gep_pattern_count)

{'GEP_t': 8700, 'GEP_u': 37261, 'GEP_n': 98703, 'pan_OG': 162502}


In [21]:
gep_pattern_count_by_strain = {
    '874': {'GEP_t': 0, 'GEP_u': 0, 'GEP_n': 0, 'pan_OG': 0},
    '914': {'GEP_t': 0, 'GEP_u': 0, 'GEP_n': 0, 'pan_OG': 0}
}

# Loop through each strain
for strain in ['874', '914']:
    # Filter data for the current strain
    strain_data = merged_data[merged_data['strain'] == strain]
    
    # Count occurrences of each GEP pattern
    gep_pattern_count_by_strain[strain]['GEP_t'] = strain_data['assigned_orthogroup'].str.count(r'GEP_t\w+').sum()
    gep_pattern_count_by_strain[strain]['GEP_u'] = strain_data['assigned_orthogroup'].str.count(r'GEP_u\w+').sum()
    gep_pattern_count_by_strain[strain]['GEP_n'] = strain_data['assigned_orthogroup'].str.count(r'GEP_n\w+').sum()
    
    # Calculate the total assigned and pan_OG counts
    total_assigned = len(strain_data['assigned_orthogroup'])
    total_known_patterns = (
        gep_pattern_count_by_strain[strain]['GEP_t'] + 
        gep_pattern_count_by_strain[strain]['GEP_u'] + 
        gep_pattern_count_by_strain[strain]['GEP_n']
    )
    gep_pattern_count_by_strain[strain]['pan_OG'] = total_assigned - total_known_patterns

# Print the counts by strain
print(gep_pattern_count_by_strain)

{'874': {'GEP_t': 3829, 'GEP_u': 18214, 'GEP_n': 51780, 'pan_OG': 76626}, '914': {'GEP_t': 4871, 'GEP_u': 19047, 'GEP_n': 46923, 'pan_OG': 85876}}


In [71]:
unique_assigned_orthogroups = merged_data['assigned_orthogroup'].nunique()
print("Total Unique Assigned Orthogroups:", unique_assigned_orthogroups)

Total Unique Assigned Orthogroups: 178986
